In [1]:
#formats the competitionData into tfRecords for RNN training, including blockwise feature normalization
baseDir = '/oak/stanford/groups/shenoy/fwillett/speechPaperRelease_final'

import os
os.makedirs(baseDir+'/derived/tfRecords', exist_ok=True)

In [ ]:
from makeTFRecordsFromSession import makeTFRecordsFromCompetitionFiles 
from getSpeechSessionBlocks import getSpeechSessionBlocks
blockLists = getSpeechSessionBlocks()

for sessIdx in range(27):
    sessionName = blockLists[sessIdx][0]
    dataPath = baseDir + '/competitionData'
    tfRecordFolder = baseDir + '/derived/tfRecords/'+sessionName
    makeTFRecordsFromCompetitionFiles(sessionName, dataPath, tfRecordFolder)